In [58]:
!pip3 install autokeras 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 24.4 MB/s 
     |████████████████████████████████| 135 kB 62.2 MB/s 
     |████████████████████████████████| 1.6 MB 82.4 MB/s 


In [59]:
from pprint import pprint

import sklearn.datasets
import sklearn.metrics
import pandas as pd
import numpy as np

import autokeras as ak

LOADING DATA

In [60]:
TrainDataSet= pd.read_csv('../content/input/tweets.csv')
TrainDataSet.head()


,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


EXPLORATORY DATA ANALYSIS OF TW

In [61]:
Grouped_Disaster = TrainDataSet.groupby(['target'])['id'].count().reset_index()
labels = ['Disaster','Non-Disaster']
print(Grouped_Disaster)


   target    id
0       0  9256
1       1  2114


In [62]:
TrainDataSet['tweetlength'] = TrainDataSet['text'].apply(lambda x:  len(str(x)))
TrainDataSet.head(5)

,id,keyword,location,text,target,tweetlength
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1,125
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1,131
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1,63
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1,87
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0,140


In [63]:
TrainDataSet['wordcount'] = TrainDataSet['text'].apply(lambda x:  len(str(x).split()))
TrainDataSet.head(5)

,id,keyword,location,text,target,tweetlength,wordcount
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1,125,19
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1,131,23
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1,63,7
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1,87,8
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0,140,23


DATA CLEANING

In [64]:
TrainDataSet['text'] = TrainDataSet['text'].apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)))
TrainDataSet.head(3)

,id,keyword,location,text,target,tweetlength,wordcount
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1,125,19
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1,131,23
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership,1,63,7


In [65]:
#REMOVE # FROM TW
TrainDataSet['text'] = TrainDataSet['text'].apply(lambda x: re.sub(r'#', '', str(x)))
TrainDataSet.head(3)

,id,keyword,location,text,target,tweetlength,wordcount
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1,125,19
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1,131,23
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership,1,63,7


In [66]:
#REMOVE HTML tags FROM TW
TrainDataSet['text'] = TrainDataSet['text'].apply(lambda x: re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', str(x)))
TrainDataSet.tail(3)

,id,keyword,location,text,target,tweetlength,wordcount
11367,11367,wrecked,🇵🇭,i feel directly attacked 💀 i consider moonbin ...,0,139,23
11368,11368,wrecked,auroraborealis,"ok who remember ""outcast"" nd the ""dora"" au?? T...",0,105,17
11369,11369,wrecked,NaN,Jake Corway wrecked while running 14th at IRP.,1,46,8


In [86]:
TrainDataSet.head(20)

,id,keyword,location,text,target,tweetlength,wordcount
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1,125,19
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1,131,23
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership,1,63,7
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership,1,87,8
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0,140,23
5,5,ablaze,OC,"If this child was Chinese, this tweet would ha...",0,122,23
6,6,ablaze,"London, England",Several houses have been set ablaze in Ngemsib...,1,139,21
7,7,ablaze,Bharat,Asansol: A BJP office in Salanpur village was ...,1,131,24
8,8,ablaze,"Accra, Ghana","National Security Minister, Kan Dapaah's side ...",0,135,18
9,9,ablaze,Searching,This creature who’s soul is no longer clarent ...,0,127,18


AUTOML MODEL

In [81]:
import tensorflow as tf
import os
from sklearn.datasets import load_files

In [94]:
#separo datps de las etiquetas de los resultados


x = TrainDataSet.drop(['target'],axis=1)

y = np.array(TrainDataSet['target'])


#separo datos en trainig y test
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y, test_size = 0.2)

x_train = np.array(X_train)
y_train = np.array(y_train)
x_test = np.array(X_test)
y_test = np.array(y_test)



print(x_train,y_train)

[[7905 'oil%20spill' 'Mexico City, Mexico'
  'Gov. Wolf vows to veto bill loosening rules for conventional oil and gas wells '
  102 15]
 [6336 'heat%20wave' nan
  'Those who claim current heat wave is unprecedented clearly have not bothered to do any research, as the attached…'
  113 19]
 [2503 'collided' "Moe's Tavern, Springfield"
  'Homer is driving drunk😱 and has collided with a funeral urn ⚱' 61 12]
 ...
 [10660 'upheaval' nan
  'Schools should brace for five years of upheaval from a triumphant party with Gove at its heart '
  118 18]
 [1559 'bombed' nan 'Who bombed the exhibit at the sea 😂 😂 😂 Maraga' 46
  11]
 [4505 'drowning' nan
  '“Sometimes we need to disconnect to reconnect. Taking a break is necessary to save yourself and others from drowning.” —…'
  121 20]] [0 0 0 ... 0 0 0]


In [77]:
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3]) 

(9096, 6)
(9096,)
[0 0 0]


In [97]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=3
)  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(x_train, y_train, epochs=10)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 3 Complete [00h 00m 16s]
val_accuracy: 0.8294444680213928

Best val_accuracy So Far: 0.8294444680213928
Total elapsed time: 00h 00m 39s
Epoch 1/10
285/285 [==============================] - 1s 2ms/step - loss: 0.5086 - accuracy: 0.7830
Epoch 2/10
285/285 [==============================] - 1s 2ms/step - loss: 0.4756 - accuracy: 0.8130
Epoch 3/10
285/285 [==============================] - 1s 2ms/step - loss: 0.4725 - accuracy: 0.8130
Epoch 4/10
285/285 [==============================] - 1s 2ms/step - loss: 0.4702 - accuracy: 0.8130
Epoch 5/10
285/285 [==============================] - 1s 2ms/step - loss: 0.4681 - accuracy: 0.8130
Epoch 6/10
285/285 [==============================] - 0s 2ms/step - loss: 0.4660 - accuracy: 0.8130
Epoch 7/10
285/285 [==============================] - 0s 2ms/step - loss: 0.4641 - accuracy: 0.8130
Epoch 8/10
285/285 [==============================] - 1s 2ms/step - loss: 0.4623 - accuracy: 0.8130
Epoch 9/10
285/285 [==============================] - 1s 2

evaluar red

In [104]:
res = clf.evaluate(x_test,y_test)

72/72 [==============================] - 0s 1ms/step - loss: 0.4633 - accuracy: 0.8184


Predecir nuevos resultados

In [103]:
y_pred = clf.predict(x_test)

for i in range(50):
  print("valor real: ", y_test[i])
  print("valor estimado: ", y_pred[i])
  print("---------------------------------- ")

72/72 [==============================] - 1s 12ms/step
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------- 
valor real:  1
valor estimado:  [0.]
---------------------------------- 
valor real:  0
valor estimado:  [0.]
---------------------------------